In [5]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import time
from tqdm import tqdm

import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By

In [2]:
kbo = pd.read_csv('../dataset/kbo/kbopitchingdata.csv')
kbo

,id,year,team,average_age,runs_per_game,wins,losses,win_loss_percentage,ERA,run_average_9,...,hit_batter,balks,wild_pitches,batters_faced,WHIP,hits_9,homeruns_9,walks_9,strikeouts_9,strikeout_walk
0,1,2021,LG Twins,26.3,3.90,72,57,0.558,3.57,3.96,...,97,5.0,43.0,5416,1.312,8.0,0.6,3.9,7.6,1.96
1,2,2021,KT Wiz,28.4,4.06,75,59,0.560,3.67,4.17,...,42,1.0,56.0,5359,1.316,8.4,0.6,3.5,7.5,2.16
2,3,2021,Doosan Bears,27.5,4.57,70,65,0.519,4.28,4.66,...,73,7.0,51.0,5596,1.487,9.2,0.7,4.2,7.4,1.77
3,4,2021,Samsung Lions,28.8,4.57,75,59,0.560,4.29,4.70,...,51,3.0,56.0,5496,1.450,9.3,0.9,3.8,7.4,1.96
4,5,2021,NC Dinos,27.7,4.80,67,67,0.500,4.50,4.95,...,77,8.0,74.0,5575,1.476,9.1,0.9,4.2,7.5,1.79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318,319,1982,OB Bears,26.5,3.98,56,24,0.700,3.19,3.95,...,33,NaN,NaN,3020,1.260,7.8,0.5,3.6,3.9,1.10
319,320,1982,MBC Blue Dragons,25.9,4.38,46,33,0.582,3.51,4.45,...,47,NaN,NaN,2994,1.298,8.4,0.7,3.3,5.0,1.52
320,321,1982,Lotte Giants,26.3,4.81,31,49,0.388,3.95,4.86,...,29,NaN,NaN,3080,1.435,9.6,0.9,3.3,4.6,1.39
321,322,1982,Haitai Tigers,23.5,4.85,38,42,0.475,3.76,4.93,...,39,NaN,NaN,3067,1.403,8.9,0.6,3.8,4.4,1.18


In [3]:
kbo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 323 entries, 0 to 322
Data columns (total 34 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   323 non-null    int64  
 1   year                 323 non-null    int64  
 2   team                 323 non-null    object 
 3   average_age          323 non-null    float64
 4   runs_per_game        323 non-null    float64
 5   wins                 323 non-null    int64  
 6   losses               323 non-null    int64  
 7   win_loss_percentage  323 non-null    float64
 8   ERA                  323 non-null    float64
 9   run_average_9        323 non-null    float64
 10  games                323 non-null    int64  
 11  games_started        184 non-null    float64
 12  games_finished       184 non-null    float64
 13  complete_game        323 non-null    int64  
 14  shutouts             323 non-null    int64  
 15  saves                323 non-null    int

In [ ]:
# driver = webdriver.Chrome()
# url = 'http://www.statiz.co.kr/stat.php?opt=0&sopt=0&re=1&ys=1983&ye=1988&se=0&te=&tm=&ty=0&qu=auto&po=0&as=&ae=&hi=&un=&pl=&da=1&o1=WAR&o2=OutCount&de=1&lr=0&tr=&cv=&ml=1&sn=30&si=&cn='
# driver.get(url)

In [6]:
url = 'http://www.statiz.co.kr/stat.php?opt=0&sopt=0&re=1&ys=1983&ye=1988&se=0&te=&tm=&ty=0&qu=auto&po=0&as=&ae=&hi=&un=&pl=&da=1&o1=WAR&o2=OutCount&de=1&lr=0&tr=&cv=&ml=1&sn=30&si=&cn='
res = requests.get(url)
res

<Response [200]>

In [63]:
html = 'https://www.koreabaseball.com/Record/Player/PitcherBasic/Basic1.aspx'
data = pd.read_html(html)
data

[Empty DataFrame
 Columns: [순위, 선수명, 팀명, ERA, G, W, L, SV, HLD, WPCT, IP, H, HR, BB, HBP, SO, R, ER, WHIP]
 Index: []]

In [12]:
html = res.text
soup = BeautifulSoup(html, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <meta charset="utf-8"/>
  <title>
   STATIZ
  </title>
  <!-- Tell the browser to be responsive to screen width -->
  <!-- meta content="width=device-width, initial-scale=1, maximum-scale=1, user-scalable=no" name="viewport" -->
  <!-- Bootstrap 3.3.5 -->
  <link href="bootstrap/css/bootstrap.min.css" rel="stylesheet"/>
  <!-- Font Awesome -->
  <!--<link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/font-awesome/4.4.0/css/font-awesome.min.css">-->
  <!--<link rel="stylesheet" href="https://cdn.jsdelivr.net/fontawesome/4.7.0/css/font-awesome.min.css">-->
  <link href="css/font-awesome-4.7.0/css/font-awesome.min.css" rel="stylesheet"/>
  <!-- Ionicons -->
  <link href="https://code.ionicframework.com/ionicons/2.0.1/css/ionicons.min.css" rel="stylesheet"/>
  <!-- Theme style -->
  <link href="dist/css/AdminLTE.min.css" rel="stylesheet"/>
  <!-- AdminLTE Skins. Choose a skin from the css/skins
       folder instead of downloading all of them to re

In [41]:
soup.select('tr:nth-child(3) > td:nth-child(2) > a')

[<a href="player.php?name=%EC%84%A0%EB%8F%99%EC%97%B4&amp;birth=1963-01-10">선동열</a>,
 <a href="player.php?name=%EC%84%A0%EB%8F%99%EC%97%B4&amp;birth=1963-01-10">선동열</a>]

In [59]:
tr = soup.select('tr')[2:]

data = []
for td in tr:
    td_2 = td.select('a')
    td_text = td_2.select()
    data.append(td_text)

data


AttributeError: 'list' object has no attribute 'select'

In [61]:
soup.select('span')

[<span id="logoimg1div" style="overflow:hidden;"><img id="logoimg1" src="logop.png" style="padding:0;width:240px;height:60px;"/></span>,
 <span class="input-group-btn" style="padding:0;margin:0;">
 <button class="btn btn-flat" id="search-btn" name="search" style="margin:0;" type="submit"><i class="fa fa-search"></i>
 </button>
 </span>,
 <span style="color:white;">스탯티즈+</span>,
 <span style="color:white;">스카이박스</span>,
 <span style="color:white;">기록실</span>,
 <span style="color:white;">경기일정</span>,
 <span style="color:white;">시즌정보</span>,
 <span style="color:white;">팀정보</span>,
 <span style="color:white;">선수정보</span>,
 <span style="color:white;">스페셜</span>,
 <span style="color:white;">추가정보</span>,
 <span style="color:#5CD1E5;">OLD SITE</span>,
 <span style="color:yellow;"><i class="fa fa-user"></i> 로그인</span>,
 <span style="color:yellow;"><i class="fa fa-sign-in"></i> 회원가입</span>,
 <span class="caret"></span>,
 <span class="label" style="text-decoration:none;padding:3px;font-size:12px;